In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

import time
try:
    # Chrome起動
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    wait = WebDriverWait(driver, 10)
 
    driver.get("https://www.nicosuma.com/iphone")  # ←あなたの対象ページに変更してください


    #状態
    grade_button_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//button[.//p[text()='状態']]"
    grade_button = driver.find_element(By.XPATH, grade_button_xpath)
    grade_button.click()

    #メモリ
    memory_button_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//button[.//p[text()='ストレージ容量']]"
    memory_button = driver.find_element(By.XPATH, memory_button_xpath)
    memory_button.click()

    #スマホのフィルターもあるから、pC用を指定しないとクリックできない
    model_button_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//button[.//p[text()='モデル']]"
    model_button = driver.find_element(By.XPATH, model_button_xpath)
    model_button.click()

    # モデルリストが表示されるのを待つ
    model_items_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//button[.//p[text()='モデル']]/..//span[@class='m-product-filter-option-list-item-text']"
    wait.until(EC.presence_of_all_elements_located((By.XPATH, model_items_xpath)))
    # モデル項目の取得
    model_elements = driver.find_elements(By.XPATH, model_items_xpath)
    model_list = [el.text for el in model_elements]
    # print(model_list)

    result = pd.DataFrame()
    count=0
    for model in model_list:
        if (count == 2):
                    break
        model_name_xpath = f"//div[@class='m-product-view-filter-pc-wrapper']//span[text()='{model}']"
        model_name_button = driver.find_element(By.XPATH, model_name_xpath)
        model_name_button.click()    
        
        # メモリリストが表示されるのを待つ
        memory_items_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//button[.//p[text()='ストレージ容量']]/..//span[@class='m-product-filter-option-list-item-text']"
        wait.until(EC.presence_of_all_elements_located((By.XPATH, memory_items_xpath)))       
        # メモリ項目の取得
        memory_elements = driver.find_elements(By.XPATH, memory_items_xpath)
        memory_list = [el.text for el in memory_elements]
        # print(memory_list)

        #クリアボタン
        time.sleep(1)
        clear_button_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//div[@class='m-product-view-filter-header-clear']"
        clear_button = driver.find_element(By.XPATH, clear_button_xpath)
        driver.execute_script("window.scrollTo(0, 0);")
        wait.until(EC.presence_of_all_elements_located((By.XPATH, clear_button_xpath)))     
        clear_button.click()
        
      
        
        for memory in memory_list:
            
            model_name_xpath = f"//div[@class='m-product-view-filter-pc-wrapper']//span[text()='{model}']"
            model_name_button = driver.find_element(By.XPATH, model_name_xpath)
            model_name_button.click() 
    
            memory_name_xpath = f"//div[@class='m-product-view-filter-pc-wrapper']//span[text()='{memory}']"
            memory_name_button = driver.find_element(By.XPATH, memory_name_xpath)
            memory_name_button.click()
            
            # 情態リストが表示されるのを待つ
            grade_items_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//button[.//p[text()='状態']]/..//span[@class='m-product-view-filter-grade']"
            wait.until(EC.presence_of_all_elements_located((By.XPATH, grade_items_xpath)))       
            # メモリ項目の取得
            grade_elements = driver.find_elements(By.XPATH, grade_items_xpath)
            grade_list = [el.text for el in grade_elements]
            # print(grade_list)
            
            #クリアボタン 
            time.sleep(1)    
            clear_button_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//div[@class='m-product-view-filter-header-clear']"
            clear_button = driver.find_element(By.XPATH, clear_button_xpath)
            driver.execute_script("window.scrollTo(0, 0);")
            clear_button.click()
            
            for grade in grade_list:
               
                
                model_name_xpath = f"//div[@class='m-product-view-filter-pc-wrapper']//span[text()='{model}']"
                model_name_button = driver.find_element(By.XPATH, model_name_xpath)
                model_name_button.click() 
    
                memory_name_xpath = f"//div[@class='m-product-view-filter-pc-wrapper']//span[text()='{memory}']"
                memory_name_button = driver.find_element(By.XPATH, memory_name_xpath)
                memory_name_button.click()
                
                grade_name_xpath = f"//div[@class='m-product-view-filter-pc-wrapper']//span[text()='{grade}']"
                grade_name_button = driver.find_element(By.XPATH, grade_name_xpath)
                grade_name_button.click()
                
                
                
                
                # # タイトルのような要素を取得
                page_buttons = driver.find_elements(By.XPATH, "//ul[@class='m-pagination-ul']//li[last()-1]")

                if page_buttons:
                    page_num = page_buttons[0]
                    print("ページ番号:", page_num.text)
                    
                    for page in range(1, int(page_num.text) + 1):
                        # 商品一覧の親要素から個々の商品要素を取得
                        elements = driver.find_elements(By.XPATH, "//div[@class='m-product-grid']/div")
                        
                        for element in elements:
                            e = element
                            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".m-product-card-price")))
                            driver.execute_script("arguments[0].scrollIntoView();", element)           
                            price = e.find_element(By.XPATH, ".//p[@class='m-product-card-price']/span")
                            
                            print("model:", model, "memory:", memory, "grade:", grade, "Price:", price.text)  
                            
                            data = pd.DataFrame([{
                                "model": model,
                                "memory": memory,
                                "grade": grade,
                                "price": price.text
                            }])
                            result = pd.concat([result, data], ignore_index=True)
                            
                              
                        next_page_button = driver.find_element(By.XPATH, "//ul[@class='m-pagination-ul']//li[last()]")
                        next_page_button.click()
                        time.sleep(1)
                        driver.execute_script("window.scrollTo(0, 0);")
                        
                        
                else:
                    # print("ページ番号ボタンが存在しません")
                     # 商品一覧の親要素から個々の商品要素を取得
                    elements = driver.find_elements(By.XPATH, "//div[@class='m-product-grid']/div")
                    
                    for element in elements:
                            e = element
                            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".m-product-card-price")))
                            driver.execute_script("arguments[0].scrollIntoView();", element)           
                            price = e.find_element(By.XPATH, ".//p[@class='m-product-card-price']/span")
                            # print("価格:", price.text)
                            
                            print("model:", model, "memory:", memory, "grade:", grade, "Price:", price.text)    
                            
                            data = pd.DataFrame([{
                                "model": model,
                                "memory": memory,
                                "grade": grade,
                                "price": price.text
                            }])
                            result = pd.concat([result, data], ignore_index=True)
                
                # ページトップへスクロール
                time.sleep(1)                
                #クリアボタン
                clear_button_xpath = "//div[@class='m-product-view-filter-pc-wrapper']//div[@class='m-product-view-filter-header-clear']"
                clear_button = driver.find_element(By.XPATH, clear_button_xpath)
                driver.execute_script("window.scrollTo(0, 0);")
                clear_button.click()
        count +=1
                    


    # # ページにアクセス
    # driver.get("https://www.nicosuma.com/iphone")  # ←あなたの対象ページに変更してください

    # # タイトルのような要素を取得
    # page_num = driver.find_element(By.XPATH, "//ul[@class='m-pagination-ul']//li[5]/button")
    # print("ページ数", page_num.text)


    # for page in range(1, int(page_num.text) + 1):
    #   # 商品一覧の親要素から個々の商品要素を取得
    #   elements = driver.find_elements(By.XPATH, "//div[@class='m-product-grid']/div")
      
    #   for element in elements:
    #       try:
    #           e = element
    #           wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".m-product-card-price")))
    #           driver.execute_script("arguments[0].scrollIntoView();", element)           
    #           price = e.find_element(By.XPATH, ".//p[@class='m-product-card-price']/span")
    #           print("価格:", price.text)
    #           title= e.find_element(By.XPATH, ".//p[@class='m-product-card-title']")
    #           print("モデル:", title.text.split(" ")[0], title.text.split(" ")[1])
    #           memory = e.find_element(By.XPATH, ".//p[@class='m-product-card-price']/span")
    #           print("メモリ:", title.text.split(" ")[2])
    #           grade = e.find_element(By.XPATH, ".//div[@class='m-product-card-link-image']/div")
    #           print("グレード:", grade.text)
              
              
              
              
    #       except Exception as e:
    #           print("価格の取得に失敗:", e)

    #   next_page_button = driver.find_element(By.XPATH, "//ul[@class='m-pagination-ul']//li[last()]")
    #   next_page_button.click()
    #   # ページトップへスクロール
    #   driver.execute_script("window.scrollTo(0, 0);")


except Exception as e:
    print("処理中にエラーが発生:", e)

finally:
    driver.quit()
    print("ブラウザを終了しました")

model: iPhone 16 memory: 128GB grade: 中古Cグレード Price: ¥115,200
処理中にエラーが発生: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
	GetHandleVerifier [0x0x5c4553+62419]
	GetHandleVerifier [0x0x5c4594+62484]
	(No symbol) [0x0x402133]
	(No symbol) [0x0x408909]
	(No symbol) [0x0x40ac7a]
	(No symbol) [0x0x40acf7]
	(No symbol) [0x0x44561e]
	(No symbol) [0x0x46f46c]
	(No symbol) [0x0x440054]
	(No symbol) [0x0x46f6e4]
	(No symbol) [0x0x49087a]
	(No symbol) [0x0x46f266]
	(No symbol) [0x0x43e852]
	(No symbol) [0x0x43f6f4]
	GetHandleVerifier [0x0x834833+2619059]
	GetHandleVerifier [0x0x82fc4a+2599626]
	GetHandleVerifier [0x0x5eb0fa+221050]
	GetHandleVerifier [0x0x5db378+156152]
	GetHandleVerifier [0x0x5e1d2d+183213]
	GetHandleVerifier [0x0x5cc438+94904]
	GetHandleVerifier [0x

In [53]:
result

,model,memory,grade,price
0,iPhone 16,128GB,中古Cグレード,"¥115,200"
1,iPhone 15,128GB,中古Aグレード,"¥106,200"
2,iPhone 15,128GB,中古Aグレード,"¥101,700"
3,iPhone 15,128GB,中古Aグレード,"¥101,700"
4,iPhone 15,128GB,中古Aグレード,"¥100,800"
...,...,...,...,...
64,iPhone 15,256GB,中古Cグレード,"¥108,700"
65,iPhone 15,256GB,中古Cグレード,"¥108,700"
66,iPhone 15,256GB,中古Cグレード,"¥107,800"
67,iPhone 15,256GB,中古Cグレード,"¥107,800"
